### Some OOP for practice (will be deleted)

In [57]:
class Downloader():
    def __init__(self):
        self.name = "Pepa"
          

In [58]:
A_person = Downloader()

In [59]:
A_person.name 

'Pepa'

# Start of a nice jupyter

In [60]:
from selenium import webdriver

In [61]:
import time

In [62]:
from math import ceil

In [63]:
from bs4 import BeautifulSoup

# How is the link structured

The website is located under domain name www.sreality.cz

Then it allows a client to search real estate under specified parameters. 

We specify: renting

type: flats

location: Prague

Then it displays 20 adverts per page

Here we break down how the link is structured

In [64]:
link_base = "https://www.sreality.cz"

In [65]:
search = "/hledani/pronajem/byty/praha"

In [66]:
pg = "?strana="

According to number of matches we can calculate the number of pages from which we have to get the links

In [67]:
matches = 7448

In [68]:
pages = ceil(matches/20)

In [69]:
pages

373

Because we understand the structure of the link we can create a list of all list that we want to visit

In [70]:
all_pages = [link_base+search+pg+str(k+1) for k in range(0, pages)]

In [71]:
all_pages[::372]

['https://www.sreality.cz/hledani/pronajem/byty/praha?strana=1',
 'https://www.sreality.cz/hledani/pronajem/byty/praha?strana=373']

Now we gathered all the links to give us the link that we will later scrape.
Sreality is a dynamic page so we will use selenium <3

## Selenium

### code for one page

We visit the webpage with selenium get the html code and find all the links that lead to our desired destination.

In [72]:
driver = webdriver.Chrome("C:\\chromedriver\\chromedriver.exe")
driver.get("https://www.sreality.cz/hledani/pronajem/byty/praha")
time.sleep(2)
res1 = driver.execute_script("return document.documentElement.outerHTML")
driver.quit()

In [73]:
soup = BeautifulSoup(res1, "lxml")

In [74]:
links = soup.find_all("a", {"class":"title"})

We check that we collected 21 links because there is one sponsored link per page.
Then we print the first link that is not sponsored

In [75]:
len(links)

21

In [78]:
links[1]['href']

'/detail/pronajem/byt/2+kk/praha-zizkov-blahnikova/181878364'

## for many pages

Now this code is generalized for visiting all the pages that we previously set out to get links from

In [79]:
driver = webdriver.Chrome("C:\\chromedriver\\chromedriver.exe")
all_links = []
for k in range(2): # when ready replace 2 with variable "pages"
    driver.get(all_pages[k])
    time.sleep(1)
    res = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(res, "lxml")
    links = soup.find_all("a", {"class":"title"})
    for j in links[1:]:
        all_links.append(j['href'])
driver.quit()

In [80]:
len(all_links)

40

In [81]:
all_links

['/detail/pronajem/byt/1+1/praha-smichov-plzenska/1797537372',
 '/detail/pronajem/byt/2+kk/praha-zizkov-blahnikova/181878364',
 '/detail/pronajem/byt/2+kk/praha-zizkov-blahnikova/3836165724',
 '/detail/pronajem/byt/3+kk/praha-zizkov-blahnikova/2225552988',
 '/detail/pronajem/byt/3+kk/praha-vinohrady-slavikova/614940252',
 '/detail/pronajem/byt/4+kk/praha-zizkov-cimburkova/2275884636',
 '/detail/pronajem/byt/4+1/praha-stare-mesto-karoliny-svetle/3687136860',
 '/detail/pronajem/byt/3+kk/praha-vinohrady-slavikova/4224007772',
 '/detail/pronajem/byt/3+kk/praha-hostivar-pod-stepem/1195589212',
 '/detail/pronajem/byt/3+kk/praha-hostivar-pod-stepem/2302885468',
 '/detail/pronajem/byt/3+kk/praha-nove-mesto-masarykovo-nabrezi/2820947548',
 '/detail/pronajem/byt/2+kk/praha-nusle-sdruzeni/4108926556',
 '/detail/pronajem/byt/1+kk/praha-vysocany-parikova/2032418396',
 '/detail/pronajem/byt/2+kk/praha-horni-mecholupy-modenska/2422685276',
 '/detail/pronajem/byt/1+kk/praha-horni-mecholupy-mantovska/2

This is just a begining of all the links we can get :D

# Now we need to decide what we want from individual sites.

In [83]:
link_explore = link_base + all_links[0]
driver = webdriver.Chrome("C:\\chromedriver\\chromedriver.exe")
driver.get(link_explore)
gist = driver.execute_script("return document.documentElement.outerHTML")

In [84]:
info = BeautifulSoup(gist, "lxml")

## magic from now on 

In [98]:
info.find_all("li")[13].find("label").get_text()

'Komunikace:'

In [94]:
info.find_all("li")[13].find("strong").get_text()

'\n\nAsfaltová\n\n\n\n\n'

In [163]:
lists = info.find_all("li")
dict = {}

for k in lists:
    try:
        column = k.find("label").get_text()
    except:
        break
    value = k.find("strong").get_text()
    if value == "\n\n\n\n\n\n\n":
        value = k.find("strong").findChild("span",{"class": "icof icon-ok ng-scope"})["ng-if"].split(" ")[-1]
    keys = value.replace("\n","")
    keys_a = keys.replace("\xa0"," ")
    #add it to the dictionary
    dict[column] = keys_a
print(dict)   

{'Celková cena:': '13 000 Kč za měsíc, + provize RK, bez poplatků, včetně právního servisu', 'Provize:': '13000,0 Kč', 'Poznámka k ceně:': 'Kauce 13000', 'Náklady na bydlení:': '2500 bez elektřiny ( v ceně internet )', 'ID zakázky:': '010720', 'Aktualizace:': 'Dnes', 'Stavba:': 'Cihlová', 'Stav objektu:': 'Po rekonstrukci', 'Vlastnictví:': 'Osobní', 'Podlaží:': '2. podlaží', 'Užitná plocha:': '45m2', 'Telekomunikace:': 'Telefon, Internet, Kabelová televize', 'Doprava:': 'Vlak, Silnice, MHD, Autobus', 'Komunikace:': 'Asfaltová', 'Vybavení:': "'boolean-true'"}


In [164]:
dict

{'Celková cena:': '13 000 Kč za měsíc, + provize RK, bez poplatků, včetně právního servisu',
 'Provize:': '13000,0 Kč',
 'Poznámka k ceně:': 'Kauce 13000',
 'Náklady na bydlení:': '2500 bez elektřiny ( v ceně internet )',
 'ID zakázky:': '010720',
 'Aktualizace:': 'Dnes',
 'Stavba:': 'Cihlová',
 'Stav objektu:': 'Po rekonstrukci',
 'Vlastnictví:': 'Osobní',
 'Podlaží:': '2. podlaží',
 'Užitná plocha:': '45m2',
 'Telekomunikace:': 'Telefon, Internet, Kabelová televize',
 'Doprava:': 'Vlak, Silnice, MHD, Autobus',
 'Komunikace:': 'Asfaltová',
 'Vybavení:': "'boolean-true'"}

it is time to test it on multiple pages...

Plus, it should be rewriten as at least functions maybe OOP